# Calculate daily min, mean, max, and mean temperatures
In this notebook, building on the methods covered in the Intro to Annual Consumption Models notebook on reproducing an annual consumption model, we find the daily min, max, mean temperature at target grid cells and/or forecast zones. 

## Step 0: Set-up 
Load in the libraries, and define the climakitae `app` object.

In [ ]:
import pandas as pd
from utils_notebook_1 import *
from climakitae.utils import get_closest_gridcell
from climakitae.selectors import Boundaries
import climakitae as ck

In [ ]:
app = ck.Application()

## Step 1: Read in the data 
For this notebook, we'll read in daily data instead of hourly data. 

In [ ]:
daily_data = app.retrieve("data/config_daily_data.csv") 

## Step 2: Read in the station data
Like in the intro notebook, we'll demo how to do this for the Sacramento weather station.

In [ ]:
stations_df = pd.read_csv("data/CEC_Forecast_Weather Stations_California.csv", index_col="STATION")
station_name = "SACRAMENTO EXECUTIVE AIRPORT"
one_station = stations_df.loc[station_name]

## Step 3: Find the daily min, max, mean values for each data variable

### 3a) Decide if you want to use the closest grid cell, or aggregate to the DFZ 
We reccommend aggregating the data to the nearest demand forecast zone, but we've also provided the code to use the single closest grid cell to the station instead. 

In [ ]:
#method = "closest grid cell" 
method = "aggregated DFZ" 

### 3b) Compute the daily statistics using your method of choice
Here, we show you how to compute the median across the demand forecast zone, but you can also aggregate by min, max, or mean-- see the intro notebook for more details. 

In [ ]:
# Clip data to get just the single closest grid cell
if method == "closest grid cell": 
    daily_data_clipped = get_closest_gridcell(daily_data, one_station.LAT_Y, one_station.LON_X)

# Clip data to the demand forecast zone, aggregate across all grid cells 
elif method == "aggregated DFZ": 
    dfzs_df = Boundaries()._ca_forecast_zones # Load geometries from catalog
    daily_data_clipped = clip_data_to_dfz(daily_data, dfzs_df, one_station.LAT_Y, one_station.LON_X)
    daily_data_clipped = daily_data_clipped.median(dim=["x","y"])

## Step 4: Output the data to csv files 
First, we'll load the data into memory. Then, we will convert the xarray dataset to a pandas dataframe object. Lastly, we will output the dataframe to a csv file.

In [ ]:
%%time
daily_data_clipped = app.load(daily_data_clipped)

In [ ]:
daily_df = daily_data_clipped.isel(scenario=0).drop(["Lambert_Conformal","scenario"]).to_dataframe()
daily_df.head()

In [ ]:
filename = "daily_2m_temp_{0}.csv".format(station_name.replace(" ", "_").lower())
daily_df.to_csv(filename, index=True)